In [0]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-0hd3jebe
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-0hd3jebe
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.06b75bbf3f3ec2634888f28094b819d88553b2c44_-cp36-none-any.whl size=114006 sha256=a446701d8a196c4a8bfe4950189ddde3af753187e516855f7282194133ab0f57
  Stored in directory: /tmp/pip-ephem-wheel-cache-1kuxxzj8/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [0]:
!wget https://snap.stanford.edu/snappy/release/snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6.tar.gz
!tar zxvf snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6.tar.gz
%cd snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6
!sudo python3 setup.py install

--2020-06-04 03:25:12--  https://snap.stanford.edu/snappy/release/snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11112593 (11M) [application/x-gzip]
Saving to: ‘snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6.tar.gz’

snap-stanford-5.0.0 100%[===================>]  10.60M  3.85MB/s    in 2.8s    

2020-06-04 03:25:15 (3.85 MB/s) - ‘snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6.tar.gz’ saved [11112593/11112593]

snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/
snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/snap.py
snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/PKG-INFO
snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/setup.py
snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/_snap.so
snap-stanford-5.0.0-5.0-ubuntu18.04.2-x64-py3.6/snap_stanford.egg-info/
snap-stanford-5.0.0-5.0-

In [0]:
import numpy as np
import pandas as pd
import os, json
import glob

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import progressbar

import matplotlib.pyplot as plt
import snap

import datetime

%load_ext tensorboard

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# dataframe = pd.read_json('../scraper/posts.json')

# posts_data = dataframe.drop(['post_id', 'thread_id', 'user_name', 'msg', 'reply_to'], axis=1)

pd.set_option('display.max_columns', None)

temp = pd.DataFrame()

path_to_json = './lihkg_data/*' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

print("Loading JSON File")

result = []
for f in progressbar.progressbar(file_list):
    with open(f, "rb") as infile:
        result = result + json.load(infile)

posts_data = pd.DataFrame(result)

posts_data.to_pickle("data_final.pkl")
posts_data.head()

- |#                                                  | 0 Elapsed Time: 0:00:00


Loading JSON File


""


In [0]:
posts_data.size

0

In [0]:
posts_data = pd.read_pickle("/content/drive/My Drive/comp4641/data_final_with_graph.pkl")

In [0]:
groups = posts_data.groupby(['user_id'], as_index=False).filter(lambda group: group['user_id'].count() > 10)

In [0]:
posts = [pd.DataFrame(y) for x, y in groups.groupby(['user_id'], as_index=False)]

In [0]:
len(posts)

53382

In [0]:
data_for_export = []

for user in progressbar.progressbar(posts):
    inner_html = ""
    for index, row in user.sort_values('create_time').iterrows():
        if row['root']:
            inner_html += '<div style="width: 100%; background-color: #ffecb8; border: solid 1px #ffd76b; border-radius: 5px; padding: 7px; margin: 10px 0;">' + row['msg'].replace("\n", "<br/>") + '</div>'
        else: 
            inner_html += '<div style="width: 100%; background-color: #ffffff; border: solid 1px #333333; border-radius: 5px; padding: 7px; margin: 10px 0;">' + row['msg'].replace("\n", "<br/>") + '</div>'
    
    result = {
        'data': {
            'user_id': user['user_id'].iloc[0],
#             'thread_url': '<a href="https://lihkg.com/thread/' + user['thread_id'].iloc[0] + '" target="blank" rel="noopener noreferrer">Thread ' + user['thread_id'].iloc[0] + '</a>',
            'display_html': '<div style="width: 100%;">' + inner_html + '</div>'
        }
    }
    data_for_export.append(result);

# with open('labeling.json', 'w') as outfile:
#     json.dump(data_for_export, outfile)

100% (53382 of 53382) |##################| Elapsed Time: 0:14:56 Time:  0:14:56


In [0]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

chunk = list(chunks(data_for_export, 10))
        
for i in range(10):
    with open('labeling' + str(i) + '.json', 'w') as outfile:
        json.dump(chunk[i], outfile)

In [0]:
def chunks(l, n):
    """Yield n number of striped chunks from l."""
    for i in range(0, n):
        yield l[i::n]

# with open('labeling.json', "rb") as infile:
#         result = json.load(infile)

chunk = list(chunks(data_for_export, 10))
        
for i in progressbar.progressbar(range(10)):
    with open('chunks/labeling_' + str(i) + '.json', 'w') as outfile:
        json.dump(chunk[i], outfile)


In [0]:
# import networkx as nx
import json
import snap

# G = nx.DiGraph()

groups = posts_data.groupby(['user_id'], as_index=False).filter(lambda group: group['user_id'].count() > 5)

G = snap.TNGraph.New()

# with open('../scraper/posts.json') as f:
#   posts = json.load(f)

# for index, post in progressbar.progressbar(posts_data.iterrows()):
#     # print(post)
#     if post['root'] is True:
#         G.add_node(post['user_id'])
#     else:
#         if post['is_not_push_post'] is True:
#             G.add_edge(post['user_id'], post['reply_to'], weight=0.3)
#         else:
#             G.add_edge(post['user_id'], post['reply_to'], weight=1)

for index, post in progressbar.progressbar(groups.iterrows()):
      if post['root'] is True:
        try:
          G.AddNode(post['user_id'])
        except:
          pass
      else:
          try:
            G.AddNode(int(post['user_id']))
            G.AddNode(int(post['reply_to']))
          except:
            pass
          try:
            G.AddEdge(int(post['user_id']), int(post['reply_to']))
          except:
            pass
        
snap.SaveEdgeList(G, '/content/drive/My Drive/Colab Notebooks/mygraph.txt')  
# hub, aut = nx.hits(G)

# ac = nx.eigenvector_centrality(G)
# cluster = nx.clustering(G)
# betweenness = nx.betweenness_centrality(G)

# nx.write_gexf(G, "lihkg.gexf")

| |                                        #    | 7483030 Elapsed Time: 0:51:22


In [0]:
G1 = snap.LoadEdgeList(snap.PUNGraph, '/content/drive/My Drive/Colab Notebooks/mygraph.txt', 0, 1, '\t')

# G1 = snap.ConvertGraph(snap.PUNGraph, G)

ac = snap.TIntFltH()
snap.GetEigenVectorCentr(G1, ac)

hub = snap.TIntFltH()
aut = snap.TIntFltH()
snap.GetHits(G1, hub, aut)

cluster = snap.TFltPrV()
result = snap.GetClustCfAll(G1, cluster)

final = {
    'ac': ac,
    'hub': hub,
    'aut': aut,
    'cluster': cluster
}


In [0]:
# with open('/content/drive/My Drive/Colab Notebooks/comp4641/data.pickle', 'wb') as handle:
#     pickle.dump(final, handle)

final['ac']

In [0]:
hub = {int(item):hub[item] for item in hub}
aut = {int(item):aut[item] for item in aut}

ac = {int(item):ac[item] for item in ac}
# cluster = {int(item.GetVal1()):item.GetVal2() for item in cluster}
# betweenness = {int(k):v for k,v in betweenness.items()}

posts_data['user_id'] = posts_data['user_id'].astype(int)

posts_data['hub_score'] = posts_data['user_id'].map(hub)
posts_data['aut_score'] = posts_data['user_id'].map(aut)
posts_data['eigenvector_centrality'] = posts_data['user_id'].map(ac)
# posts_data['clustering'] = posts_data['user_id'].map(cluster)
# posts_data['betweenness_centrality'] = posts_data['user_id'].map(betweenness)

posts_data['root'] = posts_data['root'].apply(lambda x: 1 if x else 0)
posts_data['is_newbie'] = posts_data['is_newbie'].apply(lambda x: 1 if x else 0)
posts_data['is_not_push_post'] = posts_data['is_not_push_post'].apply(lambda x: 1 if x else 0)

posts_data['like_count'] = pd.to_numeric(posts_data['like_count'])
posts_data['dislike_count'] = pd.to_numeric(posts_data['dislike_count'])
posts_data['create_time'] = pd.to_datetime(posts_data['create_time'], unit='s')

posts_data = posts_data.sample(frac=1).reset_index(drop=True)
# posts_data = posts_data.drop(['user_id'], axis=1)


In [0]:
posts_data.to_pickle("/content/drive/My Drive/Colab Notebooks/comp4641/data_final_with_graph.pkl")

In [0]:
posts_data = pd.read_pickle("/content/drive/My Drive/comp4641/data_final_with_graph.pkl")

posts_data['hub_score'] = posts_data['hub_score'].astype('float32')
posts_data['aut_score'] = posts_data['aut_score'].astype('float32')
posts_data['eigenvector_centrality'] = posts_data['eigenvector_centrality'].astype('float32')

In [0]:
posts_data.dtypes

root                               int64
thread_id                         object
post_id                           object
user_id                            int64
user_name                         object
user_level                        object
is_newbie                          int64
is_not_push_post                   int64
msg                               object
like_count                         int64
dislike_count                      int64
create_time               datetime64[ns]
reply_to                          object
hub_score                        float32
aut_score                        float32
eigenvector_centrality           float32
troll                              int64
dtype: object

In [0]:
posts_data

,root,thread_id,post_id,user_id,user_name,user_level,is_newbie,is_not_push_post,msg,like_count,dislike_count,create_time,reply_to,hub_score,aut_score,eigenvector_centrality,troll
0,0,1825419,ae3320319dad49928b4d0d61dc3a91ae669bfad2,89517,KK_OVO,10,0,0,,0,0,2020-01-15 10:17:24,167548,0.012146,0.012146,0.012146,0
1,0,1716608,df1a15d2f4c5788736a9d1b0d3618e904944c954,30503,伸縮警棍,10,0,0,通常會配合警狗行動，各手足要注意,1,0,2019-11-13 11:50:39,42246,0.008963,0.008963,0.008963,0
2,0,1802410,a2c8b2933a4abdb4879cb2504e663bceb9a89c37,48083,通街都係處,10,0,1,自組軍隊？ 你去邊度買軍火？\n錢？\n銀行系統呢？,1,0,2019-12-29 16:31:57,73143,0.081878,0.081878,0.081878,0
3,0,1757204,a78fa4697693c89e6aba1944a9b4ca29cc5afbe0,114193,好dirty囉,10,0,0,推下支持你咁努力,0,0,2019-11-26 12:01:24,35892,0.006500,0.006500,0.006500,0
4,0,1361703,720734eca987d3544581bc37b1074c9a72db3769,81008,倔強柴犬,10,0,0,齊上齊落 唔好落單,0,0,2019-07-26 10:01:12,56107,0.011058,0.011058,0.011058,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619970,0,1960969,164d91fff3e08ae335f90608914b98394942a0b8,310929,大絲球,10,1,1,本身拎左佢100 倒輸100俾佢 衰大貪\n我賭100股都咁 真係好差\n掉轉諗每日犯錯都學到野,0,0,2020-04-06 16:48:06,211761,0.003560,0.003560,0.003560,0
7619971,0,1825314,7ab2941b436e64ac4466fd0fcb6391ad9b4310e2,91397,勃朗特,10,0,0,推,0,0,2020-01-15 07:55:23,48083,0.022017,0.022017,0.022017,0
7619972,0,1638834,9820cbb31610bf957b73e23f77c6b802ac4bde12,51142,mk1or4ha4eu7,10,0,0,繼續堅持大家同行,0,0,2019-10-08 13:14:58,188828,0.002573,0.002573,0.002573,0
7619973,0,1616000,934d43d180458deefd2cf260fb3bbf84445c31ff,278786,6961261671,10,1,0,唔好畀淋病spin走！！！錯重點！！！\n萬一正苦真係用緊急法推蒙面法，焦點應放在緊急法！！...,0,0,2019-10-04 01:18:05,154486,0.003863,0.003863,0.003863,0


In [0]:
KNOWN_TROLLS = ['41853','194398','76776','71393','30019','37596','63950','310982','282494','72457','326966','322121','322637','121095','169','213597','226126','245201','159448','149494','181778','324892','273582','118226','240250','316329','77788','219767','71341','12186','228352','75196','28435','25002','247732','322304','258598','133525','14941','27416','222907','48631','158008','289946','96230','40470','186800','30788','288523','254191','84585','85242','123870','89514','170281','103799','64699','49699','299923','71140','122676','61748','202464','288351','234658','277044','149978','6237','98232','69484','165028','328913','132380','146030','331780','266100','52773','275585','249366','41351','273474','39751','51615','326708','213952','1210','270563','241151','70493','335209','276916','232237','94819','78468','203248','100028','291185','95649','332154','83060','264328','244624','105890','5591','171664','23721','129619','326414','14965','51985','191271','253253','25771','153956','32583','72825','336250','58089','70734','124277','173333','155597','137604','276602','41818','162674','151378','295632','147880','314774','57035','88029','97104','216114','64295','28833','203010','264400','273157','238973','57405','28870','132128','199261','126609','65530','172349','173849','192248','261633','127675','34757','166655','228589','343059','144849']

posts_data['troll'] = posts_data['user_id'].apply(lambda x: 0 if (str(x) not in KNOWN_TROLLS) else 1)

posts_data[posts_data['troll'] == 1]

,root,thread_id,post_id,user_id,user_name,user_level,is_newbie,is_not_push_post,msg,like_count,dislike_count,create_time,reply_to,hub_score,aut_score,eigenvector_centrality,troll
205,0,1865186,403eebb244b2b7dd7d34fe84cc393b56c2eb8eea,289946,王奴lu,10,1,1,上連燈講有能用咩\n藍屍睇閪到咩,4,0,2020-02-06 04:57:59,129988,0.012885,0.012885,0.012885,1
206,0,1583123,0a826ad30c31eb6c2b31cdd750d221ad72c9da55,28435,出嚟呀豉油雞,10,0,0,一早知啦，示威者邊有帶金鏈光頭佬呀，而且打篇千字文作文味濃，差佬/建制反應極冷淡,0,0,2019-09-20 13:11:49,222970,0.040318,0.040318,0.040318,1
210,0,1554750,0056e5726f9feb4eef9fa91cb5cd9446a5640b7d,94819,卡塔庫栗,10,0,0,反正都無野唔記得\n係你無本事同想像力去諗佢可以做到咩程度\n渣ga,0,0,2019-09-09 11:30:42,24519,0.017975,0.017975,0.017975,1
315,0,1881269,5230f5075ff45c23d329781f12adb75833173300,146030,印記,10,0,0,fei seung ho,0,1,2020-02-17 08:42:22,196701,0.020849,0.020849,0.020849,1
327,0,1324870,b36f574c3d46a4f6bd3ef06226848efb5026cc54,241151,瀧野由美仔,10,0,0,佢地同泛民左膠一膽擔，成日想控制所有抗爭者變返低能仔,0,9,2019-07-16 17:26:14,16676,0.002234,0.002234,0.002234,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619423,0,1960397,8e289ffe5878caf4bfda3dd21ff9e0afbe3b6c48,213952,張崑陽打手,10,0,0,樓主全家死 大 all in,0,0,2020-04-06 07:35:37,80564,0.016148,0.016148,0.016148,1
7619489,0,1420587,a960ffae51b11f5b14870610bbbdf4ed05582950,191271,香港人改變世界,10,0,0,https://www.facebook.com/standnewshk/videos/72...,45,0,2019-08-06 06:19:33,17232,0.008762,0.008762,0.008762,1
7619706,0,1915343,4d0cfe6b9ff643bee8a3037ed3a5db01642cfec3,41818,人生可否重來,10,0,0,個女人做咩,0,0,2020-03-08 08:18:10,112622,0.043792,0.043792,0.043792,1
7619760,0,1831648,6a5fc8d58834e9dd0571e8f23f2f04fcddee84b0,149978,集思港益,10,0,0,工資冇加，錢去晒落呢些度。\n真係要時代革命，裝修下香港,0,1,2020-01-20 14:10:11,226466,0.045804,0.045804,0.045804,1


In [0]:
posts_data = posts_data.groupby(['user_id'], as_index=False).filter(lambda group: group['user_id'].count() > 5)
posts_data

,root,thread_id,post_id,user_id,user_name,user_level,is_newbie,is_not_push_post,msg,like_count,dislike_count,create_time,reply_to,hub_score,aut_score,eigenvector_centrality,troll
0,0,1825419,ae3320319dad49928b4d0d61dc3a91ae669bfad2,89517,KK_OVO,10,0,0,,0,0,2020-01-15 10:17:24,167548,0.012146,0.012146,0.012146,0
1,0,1716608,df1a15d2f4c5788736a9d1b0d3618e904944c954,30503,伸縮警棍,10,0,0,通常會配合警狗行動，各手足要注意,1,0,2019-11-13 11:50:39,42246,0.008963,0.008963,0.008963,0
2,0,1802410,a2c8b2933a4abdb4879cb2504e663bceb9a89c37,48083,通街都係處,10,0,1,自組軍隊？ 你去邊度買軍火？\n錢？\n銀行系統呢？,1,0,2019-12-29 16:31:57,73143,0.081878,0.081878,0.081878,0
3,0,1757204,a78fa4697693c89e6aba1944a9b4ca29cc5afbe0,114193,好dirty囉,10,0,0,推下支持你咁努力,0,0,2019-11-26 12:01:24,35892,0.006500,0.006500,0.006500,0
4,0,1361703,720734eca987d3544581bc37b1074c9a72db3769,81008,倔強柴犬,10,0,0,齊上齊落 唔好落單,0,0,2019-07-26 10:01:12,56107,0.011058,0.011058,0.011058,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619970,0,1960969,164d91fff3e08ae335f90608914b98394942a0b8,310929,大絲球,10,1,1,本身拎左佢100 倒輸100俾佢 衰大貪\n我賭100股都咁 真係好差\n掉轉諗每日犯錯都學到野,0,0,2020-04-06 16:48:06,211761,0.003560,0.003560,0.003560,0
7619971,0,1825314,7ab2941b436e64ac4466fd0fcb6391ad9b4310e2,91397,勃朗特,10,0,0,推,0,0,2020-01-15 07:55:23,48083,0.022017,0.022017,0.022017,0
7619972,0,1638834,9820cbb31610bf957b73e23f77c6b802ac4bde12,51142,mk1or4ha4eu7,10,0,0,繼續堅持大家同行,0,0,2019-10-08 13:14:58,188828,0.002573,0.002573,0.002573,0
7619973,0,1616000,934d43d180458deefd2cf260fb3bbf84445c31ff,278786,6961261671,10,1,0,唔好畀淋病spin走！！！錯重點！！！\n萬一正苦真係用緊急法推蒙面法，焦點應放在緊急法！！...,0,0,2019-10-04 01:18:05,154486,0.003863,0.003863,0.003863,0


In [34]:
new_data = posts_data.drop(['thread_id', 'post_id', 'user_name', 'msg', 'create_time', 'user_level'], axis=1)
# user_id, reply_to
new_data.info()

pos = len(new_data[new_data['troll'] == 1].index)

neg_df = new_data[new_data['troll'] == 0].sample(n=pos)
pos_df = new_data[new_data['troll'] == 1]

final_embed_df = pd.concat([neg_df, pos_df])
final_df = final_embed_df.drop(['user_id', 'reply_to'], axis = 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7483031 entries, 0 to 7619974
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   root                    int64  
 1   user_id                 int64  
 2   is_newbie               int64  
 3   is_not_push_post        int64  
 4   like_count              int64  
 5   dislike_count           int64  
 6   reply_to                object 
 7   hub_score               float32
 8   aut_score               float32
 9   eigenvector_centrality  float32
 10  troll                   int64  
dtypes: float32(3), int64(7), object(1)
memory usage: 599.5+ MB


In [35]:
G = snap.TNGraph.New()

# with open('../scraper/posts.json') as f:
#   posts = json.load(f)

# for index, post in progressbar.progressbar(posts_data.iterrows()):
#     # print(post)
#     if post['root'] is True:
#         G.add_node(post['user_id'])
#     else:
#         if post['is_not_push_post'] is True:
#             G.add_edge(post['user_id'], post['reply_to'], weight=0.3)
#         else:
#             G.add_edge(post['user_id'], post['reply_to'], weight=1)

for index, post in progressbar.progressbar(final_embed_df.iterrows()):
      if post['root'] is True:
        try:
          G.AddNode(post['user_id'])
        except:
          pass
      else:
          try:
            G.AddNode(int(post['user_id']))
            G.AddNode(int(post['reply_to']))
          except:
            pass
          try:
            G.AddEdge(int(post['user_id']), int(post['reply_to']))
          except:
            pass
        
snap.SaveEdgeList(G, '/content/drive/My Drive/Colab Notebooks/new_mygraph.txt')

| |         #                                    | 155763 Elapsed Time: 0:01:05


In [0]:
train, test = train_test_split(final_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=64):
  dataframe = dataframe.copy()
  labels = dataframe.pop('troll')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
# total = len(new_data.index)

# neg = len(new_data[new_data['troll'] == 0].index)
# pos = len(new_data[new_data['troll'] == 1].index)

# weight_for_0 = (1 / neg)*(total)/2.0 
# weight_for_1 = (1 / pos)*(total)/2.0

# class_weight = {0: weight_for_0, 1: weight_for_1}

# print('Weight for class 0: {:.2f}'.format(weight_for_0))
# print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [0]:
feature_columns = []

# numeric cols
# for header in ['user_level', 'like_count', 'dislike_count', 'hub_score', 'aut_score', 'eigenvector_centrality', 'clustering', 'betweenness_centrality']:
for header in ['like_count', 'dislike_count', 'hub_score', 'aut_score', 'eigenvector_centrality']:
  feature_columns.append(feature_column.numeric_column(header))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'is_newbie', [0, 1])))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'is_not_push_post', [0, 1])))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'root', [0, 1])))


In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [0]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of targets:', label_batch )

In [0]:
# from tensorflow.keras import regularizers

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['acc'])

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [0]:
%tensorboard --logdir logs

In [0]:
history = model.fit(train_ds,
          validation_data=val_ds,
          epochs=100, callbacks=[early_stop,tensorboard_callback])

In [0]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/comp4641/saved_model')

In [0]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

labels = [];
for feature_batch, label_batch in test_ds.take(-1):
  labels.append(label_batch)

test_prod = model.predict(test_ds)
plot_cm(tf.concat(labels,0), tf.convert_to_tensor(test_prod, dtype=tf.float32))


In [0]:
df_by_time = posts_data.set_index('create_time').groupby(pd.Grouper(freq='1D', label='right'))

# # df_by_time['troll'] = posts_data[posts_data['troll'] == 1]['create_time']
# df_by_time.index = posts_data[posts_data['troll'] == 0]['create_time']

In [0]:
df_by_time.first()

In [0]:
plot_df = pd.DataFrame()

plot_df['count'] = df_by_time.count()['post_id']
plot_df['troll'] = df_by_time['troll'].agg('sum')

In [0]:
plt.figure(figsize=(20,10))
ax = plot_df['count'].plot()
ax = plot_df['troll'].plot(secondary_y=True,color='r')